In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage

model = init_chat_model(model="gpt-4o-mini", temperature=0.0)

model.invoke([HumanMessage(content="Hi I'm Pon")])

AIMessage(content='Hi Pon! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BtgbkiRL0zisKAvKmx3F2XzOVVnZX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--54d207b9-a4d7-49c7-8c1f-ff0d24c10fe0-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
model.invoke([HumanMessage(content="What is my name?")])

AIMessage(content="I don't have access to personal information about users unless it has been shared with me in the course of our conversation. If you'd like to share your name or any other details, feel free to do so!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 12, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BtgcH4uAaWoVMznovr75RMGVjxFLj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c674dfb0-486f-4438-a7ca-20d6f6964f7f-0', usage_metadata={'input_tokens': 12, 'output_tokens': 41, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
model.invoke(
   [
      HumanMessage(content="Hi I'm Pon"),
      AIMessage(content="Hello Pon! How can I assist you today?"),
      HumanMessage(content="What is my name?"), 
   ]
)

AIMessage(content='Your name is Pon. How can I help you today, Pon?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BtgeKHHrmHCfkGc4krgvjZMRcXHBF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7024114b-ac9e-454e-a9f5-1d675b3b7cba-0', usage_metadata={'input_tokens': 33, 'output_tokens': 14, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Message persistence

In [31]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState) -> MessagesState: 
   response = model.invoke(state["messages"])
   return {"messages": response}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")


memory  = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [32]:
config = {"configurable": {"thread_id": "abc123"}}

In [33]:
query = "Hi I'm Pon"

input_messages = [HumanMessage(content=query)]
output = app.invoke({"messages": input_messages}, config=config)

In [34]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Pon! How can I assist you today?


In [35]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Pon. How can I help you today, Pon?


Great! Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [25]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?


In [36]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

config = {"configurable": {"thread_id": "abc123"}}

# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config=config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?


### Prompt Templates

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
   [
      (
         "system",
         "You talk like a pirate. Answer all questions to the best of your ability."   
      ),  
      MessagesPlaceholder(variable_name="messages"),
   ]
)

In [41]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState) -> MessagesState:
   prompt = prompt_template.invoke(state)
   response = model.invoke(prompt)
   return {"messages": response}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory) 

In [42]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Pon."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Pon! What brings ye to these treacherous waters today? Be ye seekin' treasure, tales of adventure, or somethin' else entirely? Speak up, matey!


In [43]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ye be callin' yerself Pon, if I be rememberin' right! A fine name for a brave soul sailin' the seas of knowledge! What else can I do fer ye, me hearty?


In [48]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [49]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict): 
   messages: Annotated[Sequence[BaseMessage], add_messages]
   language: str
   
workflow = StateGraph(state_schema=State)

def call_model(state: State): 
   prompt = prompt_template.invoke(state)
   response = model.invoke(prompt)
   return {"messages": response}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [50]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Pon."
language = "Thai"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "language": language}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

สวัสดีครับ/ค่ะ ปอน! มีอะไรให้ช่วยเหลือหรือสอบถามได้เลยนะครับ/ค่ะ


### Managing Conversation History 

In [66]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [67]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [61]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. If you'd like to share it, feel free!


In [68]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

It seems you haven't asked a math problem yet. If you have one in mind, feel free to share it, and I'll be happy to help!


### Streaming

In [69]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Pon, Please tell me a joke"
language = "Thai"

input_messages = [HumanMessage(query)]
for chunk, meatadata in app.stream(
    {"messages": input_messages, "language": language}, 
    config,
    stream_mode="messages"
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")

|ส|วั|สด|ี|ครับ| ป|อน|!| น|ี่|คือ|ม|ุ|ข|ต|ล|ก|สำหรับ|คุณ|:

|ทำ|ไม|ค|อม|พ|ิว|เตอร์|ถึง|ไม่|สามารถ|ข|้าม|ถ|นน|ได้|?

|เพ|ราะ|มัน|กล|ัว|ไว|ร|ัส|!| 😄||